In [1]:
# Scikit-Learn ≥0.20 is required
import sklearn

# TensorFlow ≥2.0-preview is required
import tensorflow
assert tensorflow.__version__ >= "2.0"

#import tensorflow.keras

# Common imports
import numpy as np
import os

import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Input,Convolution2D,MaxPooling2D,Dense,Dropout,Activation,Flatten,Lambda,Reshape
from tensorflow.keras import backend as K

In [2]:
class DenseTranspose(tensorflow.keras.layers.Layer):
    def __init__(self, dense, activation=None, **kwargs):
        self.dense = dense
        self.activation = tensorflow.keras.activations.get(activation)
        super().__init__(**kwargs)
    def build(self, batch_input_shape):
        self.biases = self.add_weight(name="bias",
                                      shape=[self.dense.input_shape[-1]],
                                      initializer="zeros")
        super().build(batch_input_shape)
    def call(self, inputs):
        z = inputs @ K.transpose(self.dense.weights[0]) + self.biases
        return self.activation(z)

In [ ]:
class DenseCopy(tensorflow.keras.layers.Layer):
    def __init__(self, dense, activation=None, **kwargs):
        self.dense = dense
        self.activation = tensorflow.keras.activations.get(activation)
        super().__init__(**kwargs)
    def build(self, batch_input_shape):
        self.biases = self.add_weight(name="bias",
                                      shape=[self.dense.input_shape[-1]],
                                      initializer="zeros")
        super().build(batch_input_shape)
    def call(self, inputs):
        z = inputs @ self.dense.weights[0] + self.biases
        return self.activation(z)

In [24]:
import tensorflow.keras as keras

compassos = []

for k in range(10):
    Encoder_Inicial = Sequential([
        keras.layers.InputLayer((128,96,10)),
        keras.layers.Cropping3D(cropping=((0, 128), (0, 96), (k, k))),
        keras.layers.Flatten(),
        keras.layers.Dense(10, activation="relu")
    ])
    compassos.append(Encoder_Inicial)

Concatenado = keras.layers.Concatenate(compassos)

Encoder = Sequential([
    Concatenado,
    keras.layers.Dense(10, activation="relu")
])


ValueError: Input 0 of layer cropping3d_13 is incompatible with the layer: expected ndim=5, found ndim=4. Full shape received: [None, 128, 96, 10]

In [15]:
Decoder_Inicial = Sequential([
    DenseTranspose(Encoder.layers[1], activation="relu")
])

Split = []

for k in range(10):
    Decoder_Final = Sequential([
        keras.layers.Cropping1D(cropping=(k*10,(k+1)*10-1)),
        DenseTranspose(compassos[k].layers[2], activation="softmax"),
        keras.layers.Reshape((128, 96,1))
    ])
    
    Split.append(Decoder_Final)

Concatenado_Final = tensorflow.keras.layers.Concatenate(Split)
    
Decoder = Sequential([
    Concatenado_Final,
    Dropout(0.5)
])

In [16]:
model = Sequential([Encoder,Decoder])

In [17]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tensorflow.keras.optimizers.SGD(lr=0.1),
              metrics=["accuracy"])
model.build([])
model.summary()

Model: "sequential_61"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   


ValueError: You must provide an `input_shape` argument.